In [2]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer   # Term Frequency - Inverse Document Frequency
import spacy
import tensorflow as tf
nlp = spacy.load('en_core_web_sm')



data = pd.read_csv(
    "spam_ham_dataset.csv",
    #on_bad_lines=False,
    engine="python"     # ParserError: Error tokenizing data. C error: EOF inside string starting at row 1989
)

df = pd.DataFrame(data)
df.dropna()

X = df["text"]
y = df["label_num"]

X

0       Subject: enron methanol ; meter # : 988291\r\n...
1       Subject: hpl nom for january 9 , 2001\r\n( see...
2       Subject: neon retreat\r\nho ho ho , we ' re ar...
3       Subject: photoshop , windows , office . cheap ...
4       Subject: re : indian springs\r\nthis deal is t...
                              ...                        
5166    Subject: put the 10 on the ft\r\nthe transport...
5167    Subject: 3 / 4 / 2000 and following noms\r\nhp...
5168    Subject: calpine daily gas nomination\r\n>\r\n...
5169    Subject: industrial worksheets for august 2000...
5170    Subject: important online banking alert\r\ndea...
Name: text, Length: 5171, dtype: object

### Data Preprocessing

In [3]:
# 1. Lower
X = X.str.lower()

X

0       subject: enron methanol ; meter # : 988291\r\n...
1       subject: hpl nom for january 9 , 2001\r\n( see...
2       subject: neon retreat\r\nho ho ho , we ' re ar...
3       subject: photoshop , windows , office . cheap ...
4       subject: re : indian springs\r\nthis deal is t...
                              ...                        
5166    subject: put the 10 on the ft\r\nthe transport...
5167    subject: 3 / 4 / 2000 and following noms\r\nhp...
5168    subject: calpine daily gas nomination\r\n>\r\n...
5169    subject: industrial worksheets for august 2000...
5170    subject: important online banking alert\r\ndea...
Name: text, Length: 5171, dtype: object

In [4]:
# 2. Remove special characters
spl_chars_removed = []
for sentence in X:
  spl_chars_removed.append(re.sub('[^A-Za-z0-9+]', " ", sentence))
X = spl_chars_removed

X[0]


'subject  enron methanol   meter     988291  this is a follow up to the note i gave you on monday   4   3   00   preliminary  flow data provided by daren      please override pop   s daily volume   presently zero   to reflect daily  activity you can obtain from gas control    this change is needed asap for economics purposes  '

In [5]:
# 4. Lemmatize with Remove stopwords
import spacy
nlp = spacy.load('en_core_web_sm')

tokenized_emails = []
for text in tqdm(X):
  doc = nlp(text)   # for each email
  tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_space)]

  tokenized_emails.append(" ".join(tokens))

X = tokenized_emails


100%|█████████████████████████████████████████████| 5171/5171 [04:51<00:00, 17.76it/s]


In [6]:
nlp2 = spacy.blank('en')
doc2 = nlp2('I am fred eat\'s')

vocab = [token.text for token in nlp('I am fred let"s')]
vocab

['I', 'am', 'fred', 'let"s']

TF-IDF vectorizer takes the vocabulary into account and calculates the Word Vector

In [13]:
# 5. Vectorize
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val , y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)


vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X).toarray()    # vectorizer.transform(x) can be used for already fitted Vectorizer

#X_tfidf_array = X_tfidf.toarray()
y = np.array(y)

X_train_vectorized, X_test_vectorized, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.1, random_state=42)
X_train_vectorized, X_val_vectorized , y_train, y_val = train_test_split(X_train_vectorized, y_train, test_size=0.1)

In [15]:
X_train_vectorized.shape, y_train.shape         # (5171, 46161)  - where 46161 is the size of the vocabulary

((4187, 45992), (4187,))

In [18]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(45992,)),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)


history = model.fit(X_train_vectorized, y_train, validation_data=(X_val_vectorized, y_val), epochs=10)

Epoch 1/10
131/131 [==============================] - 6s 39ms/step - loss: 0.4578 - accuracy: 0.7738 - val_loss: 0.3080 - val_accuracy: 0.8970
Epoch 2/10
131/131 [==============================] - 2s 18ms/step - loss: 0.1960 - accuracy: 0.9764 - val_loss: 0.1647 - val_accuracy: 0.9785
Epoch 3/10
131/131 [==============================] - 2s 18ms/step - loss: 0.0948 - accuracy: 0.9947 - val_loss: 0.1088 - val_accuracy: 0.9871
Epoch 4/10
131/131 [==============================] - 2s 18ms/step - loss: 0.0536 - accuracy: 0.9983 - val_loss: 0.0827 - val_accuracy: 0.9893
Epoch 5/10
131/131 [==============================] - 3s 23ms/step - loss: 0.0339 - accuracy: 0.9988 - val_loss: 0.0681 - val_accuracy: 0.9893
Epoch 6/10
131/131 [==============================] - 2s 18ms/step - loss: 0.0230 - accuracy: 0.9993 - val_loss: 0.0600 - val_accuracy: 0.9893
Epoch 7/10
131/131 [==============================] - 2s 18ms/step - loss: 0.0165 - accuracy: 0.9998 - val_loss: 0.0543 - val_accuracy: 0.9893

In [19]:
test_loss, test_accuracy = model.evaluate(X_test_vectorized, y_test)

17/17 [==============================] - 0s 5ms/step - loss: 0.0491 - accuracy: 0.9865


### Model prediction function

In [21]:
def model_predictions(model, X_predict):
  y_preds = model.predict(X_predict).reshape((X_predict.shape[0], ))  # the rows - number of emails
  preds_spam_ham = ['spam' if prob==1 else 'ham' for prob in np.round(y_preds)]

  return preds_spam_ham


test_emails = vectorizer.transform(
  ['Dinner tonight in my house', 'free money lottery coupons now', 'youve won the lottery click the link', 'Breakfast today in John\'s house', 'The principal asked us to leave']
).toarray()

model_predictions(model, test_emails)

1/1 [==============================] - 0s 35ms/step


['ham', 'spam', 'spam', 'ham', 'ham']

In [26]:
predictions = model_predictions(model, X_test_vectorized[0:10])
print(predictions)

for i in range(len(predictions)):
    print(X_test[i][:100], ':', predictions[i])

1/1 [==============================] - 0s 30ms/step
['ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham']
subject hpl nom march 30 2001 attached file hplno 330 xls hplno 330 xls : ham
subject online pharxmacy 80 med disscount phafrmacy onlsine grasnd opegne 80 med orfder today doorst : spam
subject nom actual volume april 17 th agree eileen ponton 04 18 2001 10 48 19 david avila lsp enserc : ham
subject meter 8740 dec 99 robert head decide enter partnership agreement day question party thu real : ham
subject coastal oil gas corporation melissa deal 348450 create enter sitara addition volume edit 3 0 : ham
subject enron hpl actual july 27 2000 teco tap 25 000 enron 48 750 hpl iferc ls hpl lsk ic 15 000 en : ham
subject meter 98 9699 98 2662 receipt meter w delivery nom fuel buy sell hplc hplr meter 9699 s 2 5  : ham
subject hpl nom 3 2001 attached file hplno 503 xls hplno 503 xls : ham
subject enron hpl nom february 23 25 2001 february 23 2001 teco tap 30 000 february 